In [1]:
# !pip install -q selenium==4.3.0
# !pip install lxml
# !pip install -q beautifulsoup4==4.11.1
# !pip install -q backoff==2.1.2
# import backoff
# backoff.__version__
# !pip install -q pandas
# !pip install python-dotenv

In [2]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys # 鍵盤事件
import pandas as pd
import backoff
from datetime import datetime
import pytz


tz = pytz.timezone('Asia/Taipei')
today = datetime.now(tz)
date_ = today.strftime("%Y%m%d_%H%M")

In [3]:
def set_ua():
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
    return user_agent

In [4]:
@backoff.on_exception(backoff.expo,
                        Exception, 
                      max_time=10)
def chrome_init():
    chrome_opt = webdriver.ChromeOptions()
    # chrome_opt.add_argument('--headless')
    chrome_opt.add_argument('--no-sandbox')
    chrome_opt.add_argument('--ignore-ssl-errors=yes')
    chrome_opt.add_argument('--ignore-certificate-errors')
    chrome_opt.add_argument(f'user-agent={set_ua()}')
    # chrome_opt.add_argument("--incognito")  # 使用無痕模式。用 selenium開瀏覽器已經很乾淨了，但疑心病重的可以用一下
    driver = webdriver.Remote(
        command_executor='http://selenium-hub:4444/wd/hub',
        options=chrome_opt
    )
    
    return driver

In [5]:
# url = 'https://store.steampowered.com/search/?filter=globaltopsellers&ignore_preferences=1' # 指定網址
# url = 'https://store.steampowered.com/search/?ndl=1' # 指定網址
url = 'https://store.steampowered.com/search/?sort_by=Reviews_DESC&ndl=1' # 指定網址

driver = chrome_init()
driver.get(url) # 進入指定網址
driver.implicitly_wait(5) # 等待伺服器反應最多 5 秒，如果在時間到之前反應就提早結束等待

In [6]:
# driver.定位.send_keys(Keys.特殊鍵, Keys.特殊鍵, Keys.特殊鍵) # 通式
for i in range(700):
    driver.find_element('css selector', 'html').send_keys(Keys.END) # 通常定位在最根部的 html 即可
    sleep(1.5)
    print(i)

html_source = driver.page_source

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
# 獲取頁面源代碼
html_source = driver.page_source

# 使用 BeautifulSoup 解析 HTML 內容
soup = BeautifulSoup(html_source, 'lxml')

all = soup.find('div',  id = "search_result_container")
data = all.find_all('div', class_ = "responsive_search_name_combined")
href_ = all.find_all('a', href=True)


columns = ['GameName', 'GameID', 'Platform', 'Cost', 'OperationDate', 'Datetime']

save_lst = []
for stream, hf in zip(data, href_):

    href = hf['href']
    app_id = href.split('/')[4]

    d1 = today.strftime("%Y-%m-%d %H:%M")


    title = stream.find("span", class_ = "title").text


    costs = stream.find("div", class_ = "discount_final_price")
    if costs:
        cost = costs.text
    else:
        cost = "N/A"
    # cost = stream.find("div", class_ = "discount_final_price").text

    OperationDates = stream.find("div", class_ = "col search_released responsive_secondrow")
    if OperationDates:
        OperationDate = OperationDates.text
        OperationDate = OperationDate.replace("\n", "").strip()
    else:
        OperationDate = "N/A"

    #OperationDate = stream.find("div", class_ = "col search_released responsive_secondrow").text


    platforms = stream.find_all('span', class_='platform_img')
    platform_list = [platform['class'][1] for platform in platforms]


    
    value = [title, app_id, platform_list, cost, OperationDate, d1]
    save_lst.append(value)

df = pd.DataFrame(
    data = save_lst,
    columns = columns
)

driver.quit()


In [8]:

# df.to_csv(f'csv/steam_platform_info/steam_platform_info_{date_}.csv', index = 0, encoding = "utf-8")
df.to_csv(f'csv/steam_platform_info/steam_allgame_info_{date_}.csv', index = 0, encoding = "utf-8")